<a href="https://colab.research.google.com/github/ZinoTAT/Machine-Learning-Practices/blob/master/practices/VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

VGG16小练习，pytorch实现




In [10]:
from __future__ import absolute_import, division, print_function, unicode_literals

import torch
import torchvision as tv

import numpy as np
import matplotlib.pyplot as plt

print("torch_version",torch.__version__)

torch_version 1.3.1


In [0]:
class VGG16(torch.nn.Module):
  def __init__(self, n_class=9, h=128, w = 128):
    super(VGG16, self).__init__()
    self.ReLU = torch.nn.ReLU(inplace=True)
    self.MaxPool = torch.nn.MaxPool2d(2, stride=2, ceil_mode=True)
    #第一层
    self.conv1_1 = torch.nn.Conv2d(3, 64, 3, padding=1)
    self.conv1_2 = torch.nn.Conv2d(64, 64, 3, padding=1)
    
    #第二层
    self.conv2_1 = torch.nn.Conv2d(64, 128, 3, padding=1)
    self.conv2_2 = torch.nn.Conv2d(128, 128, 3, padding=1)

    #第三层
    self.conv3_1 = torch.nn.Conv2d(128, 256, 3, padding=1)
    self.conv3_2 = torch.nn.Conv2d(256, 256, 3, padding=1)
    self.conv3_3 = torch.nn.Conv2d(256, 256, 3, padding=1)

    #第四层
    self.conv4_1 = torch.nn.Conv2d(256, 512, 3, padding=1)
    self.conv4_2 = torch.nn.Conv2d(512, 512, 3, padding=1)
    self.conv4_3 = torch.nn.Conv2d(512, 512, 3, padding=1)

    #第五层
    self.conv5_1 = torch.nn.Conv2d(256, 512, 3, padding=1)
    self.conv5_2 = torch.nn.Conv2d(512, 512, 3, padding=1)
    self.conv5_3 = torch.nn.Conv2d(512, 512, 3, padding=1)

    #全连接层
    self.Linear1 = torch.nn.Linear((h * w) / (32 * 32) * 512, 4096)
    self.Linear2 = torch.nn.Linear(4096, 4096)
    self.Linear3 = torch.nn.Linear(4096, 1000)
    
    #softmax
    self.output = torch.nn.Linear(1000, n_class)
    self.softmax = torch.nn.softmax(n_class)
  def forward(self, x):
    h = self.ReLU(self.conv1_1(x))
    h = self.ReLU(self.conv1_2(h))
    h = self.MaxPool(h)

    h = self.ReLU(self.conv2_1(h))
    h = self.ReLU(self.conv2_2(h))
    h = self.MaxPool(h)

    h = self.ReLU(self.conv3_1(h))
    h = self.ReLU(self.conv3_2(h))
    h = self.ReLU(self.conv3_3(h))
    h = self.MaxPool(h)

    h = self.ReLU(self.conv4_1(h))
    h = self.ReLU(self.conv4_2(h))
    h = self.ReLU(self.conv4_3(h))
    h = self.MaxPool(h)

    h = self.ReLU(self.conv5_1(h))
    h = self.ReLU(self.conv5_2(h))
    h = self.ReLU(self.conv5_3(h))
    h = self.MaxPool(h)

    h = self.ReLU(self.Linear1(h))
    h = self.ReLU(self.Linear2(h))
    h = self.ReLU(self.Linear3(h))

    h = self.softmax(h)
    return h



In [14]:
module = VGG16
d_train_p = tv.datasets.MNIST('/data/',download=True)
d_train = torch.utils.data.DataLoader(d_train_p)

0it [00:00, ?it/s]

9920512it [00:01, 8620376.41it/s]                            


Extracting /data/MNIST/raw/train-images-idx3-ubyte.gz to /data/MNIST/raw


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 129966.65it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /data/MNIST/raw/train-labels-idx1-ubyte.gz to /data/MNIST/raw


1654784it [00:00, 2365968.22it/s]                           
0it [00:00, ?it/s]

Extracting /data/MNIST/raw/t10k-images-idx3-ubyte.gz to /data/MNIST/raw


8192it [00:00, 49879.71it/s]            

Extracting /data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /data/MNIST/raw
Processing...
Done!


In [17]:
print(d_train)